In [5]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
from tqdm import tqdm
import os
import json

In [2]:
url_base = 'https://vnexpress.net'
headers = ['the-thao', 'khoa-hoc', 'suc-khoe', 'giao-duc', 'du-lich']

In [ ]:
url_subpage =[]
for header in headers:
    url_post = []
    
    for page in range(1,10):
        response = requests.get(url_base+'/'+header+'-p'+str(page))
        soup = BeautifulSoup(response.text, 'html.parser')
        posts = soup.find_all(class_='title-news')
        for post in posts:
            url_post.append(post.a['href'])
    url_subpage.append({
        header: list(set(url_post))
    })

In [ ]:
with open('url.json', 'w') as f:
    json.dump(url_subpage, f, indent=4)

In [ ]:
len(url_subpage[1]['khoa-hoc'])

275

In [ ]:
%%bash
rm -r data

In [ ]:
os.makedirs('./data')
# for header in headers:
#   os.makedirs(f'./data/{header}')

In [ ]:
rag = tqdm(url_subpage)
for urls in rag:
    header =list(urls.keys())[0]
    url_post = list(urls.values())[0]
    json_result = []
    while len(json_result) <250 and url_post:
        rag.set_description(f'{header} - {len(json_result)}')
        url = url_post.pop()
        response = requests.get(url)
        try:
            soup = BeautifulSoup(response.text, 'html.parser')
            title = soup.find(class_='title-detail').text
            description = soup.find('p', class_='description')
            for i in description.find_all(class_='location-stamp'):
                i.decompose()
            description = description.text
            content = '\n'.join([p.text for p in soup.find_all('p', class_='Normal')])
            title_ = re.sub(r'[^a-zA-Z]+', '', title).lower()
            content = description+'\n'+content
            if len(content) < 1000:
                continue
            json_result.append({
                'title': title,
                'content': description+'\n'+content,
            })
        except AttributeError:
            print('Error')
            continue
        
    with open(f'./data/{header}.json', 'w') as f:
        json.dump(json_result, f, ensure_ascii=False, indent=4)
        

the-thao - 132:   0%|          | 0/5 [01:17<?, ?it/s]

Error


the-thao - 143:   0%|          | 0/5 [01:24<?, ?it/s]

Error


the-thao - 149:   0%|          | 0/5 [01:28<?, ?it/s]

...

du-lich - 230:  80%|████████  | 4/5 [13:15<02:40, 160.40s/it]

Error


du-lich - 240:  80%|████████  | 4/5 [13:24<02:40, 160.40s/it]

Error


du-lich - 249: 100%|██████████| 5/5 [13:31<00:00, 162.38s/it]


### 2. Tách từ (Tokenize) sử dụng thư viện pyvi hay underthesea

In [2]:
import underthesea
from underthesea import word_tokenize

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [6]:
files = os.listdir('./data')
paths = [file for file in files if file.endswith('.json')]

In [7]:
data = []
labels = []
for path in paths:
    with open( os.path.join('./data', path), 'r') as f:
        doc = json.load(f)
        for item in doc:
            data.append(item['content'])
            labels.append(path.split('.')[0])
            
            

In [8]:
df = pd.DataFrame({'data': data, 'label': labels})

In [9]:
df['text'] = df['data'].apply(lambda x: x.lower())

In [10]:
df['text'] = df['text'].apply(lambda x: word_tokenize(x, format="text"))

In [11]:
# from nltk.corpus import stopwords

# # Lấy danh sách stop words tiếng Việt
# stop_words = set(stopwords.words('vietnamese'))

In [12]:
# def process(text):
#     text = word_tokenize(text, format="text")
#     text = [word for word in text.split() if word not in stop_words]
#     return ' '.join(text)

In [13]:
# df['text'] = df['text'].apply(lambda x: process(x))

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1235 entries, 0 to 1234
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   data    1235 non-null   object
 1   label   1235 non-null   object
 2   text    1235 non-null   object
dtypes: object(3)
memory usage: 29.1+ KB


In [15]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split


In [16]:
X_train, X_test, y_train, y_test = train_test_split(df['text'], df['label'], test_size=.1, random_state=40)

In [24]:
lb = LabelEncoder()
y_train = lb.fit_transform(y_train)
y_test = lb.fit_transform(y_test)


In [ ]:
tfidf_vectorizer = TfidfVectorizer(max_features=5500)
tfidf_vectorizer.fit(df['text'])

X_train_idf = tfidf_vectorizer.transform(X_train)
X_test_idf = tfidf_vectorizer.transform(X_test)



In [ ]:
knn= KNeighborsClassifier(n_neighbors=10)
kf= KFold(n_splits=5, shuffle=True, random_state=42)
scores= cross_val_score(knn, X=X_train_idf, y=y_train, cv=kf)

In [ ]:
print(scores)
print(scores.mean())

[0.97757848 0.96396396 0.98648649 0.95495495 0.96846847]
0.9702904698420394


In [25]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix

In [ ]:

svm_model = SVC()
svm_model.fit(X_train_idf.toarray(), y_train)

# Đánh giá mô hình
y_pred = svm_model.predict(X_test_idf.toarray())
accuracy = accuracy_score(y_test, y_pred)
confusion_mat = confusion_matrix(y_test, y_pred)

print("Accuracy:", accuracy)
print("Confusion Matrix:\n", confusion_mat)

Accuracy: 0.9758064516129032
Confusion Matrix:
 [[25  0  2  0  0]
 [ 0 22  0  0  0]
 [ 0  0 21  0  0]
 [ 0  0  1 26  0]
 [ 0  0  0  0 27]]


In [28]:
from sklearn.naive_bayes import GaussianNB

In [ ]:
bayes_model = GaussianNB()
bayes_model.fit(X_train_idf.toarray(), y_train)

y_pred = bayes_model.predict(X_test_idf.toarray())
accuracy = accuracy_score(y_test, y_pred)
confusion_mat = confusion_matrix(y_test, y_pred)

print("Accuracy:", accuracy)
print("Confusion Matrix:\n", confusion_mat)

Accuracy: 0.9193548387096774
Confusion Matrix:
 [[24  1  2  0  0]
 [ 3 19  0  0  0]
 [ 2  0 19  0  0]
 [ 0  1  0 26  0]
 [ 1  0  0  0 26]]


In [ ]:
from sklearn.pipeline import Pipeline

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin

In [ ]:

class ProcessText(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        if isinstance(X, list):
            return self.process(X)
        return self.process([X])
        return processed_text
    def process(self, text):
        for t in text:
            t = t.lower()
            t = word_tokenize(t, format="text")
        return tfidf_vectorizer.transform(text).toarray()


In [ ]:
pipeline_predict = Pipeline([
    ('process', ProcessText()),
    ('model', svm_model)
])


predicted_class = pipeline_predict.predict("""Năm nay, hơn 660.000 thí sinh đăng ký xét tuyển đại học với 3,4 triệu nguyện vọng. Ngay sau khi kết thúc thời gian nhập học, các trường chưa tuyển đủ được tuyển bổ sung đến tháng 12.
""")
lb.inverse_transform(predicted_class)

array(['giao-duc'], dtype=object)

In [ ]:
# save model
import pickle

with open('model.pkl', 'wb') as f:
    pickle.dump(pipeline_predict, f)
    
with open('tfidf_vectorizer.pkl', 'wb') as f:
    pickle.dump(tfidf_vectorizer, f)

with open('label_encoder.pkl', 'wb') as f:
    pickle.dump(lb, f)

## Câu 2

In [7]:
from sklearn.feature_extraction.text import HashingVectorizer
corpus = [
 ' Hôm_nay tôi đi_học', 
 ' Hôm_nay tôi đi_học ở trường', 
 ' Hôm_nay tôi nghỉ ở nhà', 
 ' Hôm_nay tôi có đi_học không?',
]
vectorizer = HashingVectorizer(n_features=2**4)
X = vectorizer.fit_transform(corpus)
print(X.shape)


  (0, 340)	-0.5773502691896258
  (0, 397)	-0.5773502691896258
  (0, 451)	-0.5773502691896258
  (1, 282)	0.5
  (1, 340)	-0.5
  (1, 397)	-0.5
  (1, 451)	-0.5
  (2, 361)	0.5
  (2, 397)	-0.5
  (2, 448)	0.5
  (2, 451)	-0.5
  (3, 113)	0.4472135954999579
  (3, 340)	-0.4472135954999579
  (3, 352)	0.4472135954999579
  (3, 397)	-0.4472135954999579
  (3, 451)	-0.4472135954999579


## Câu 3

In [17]:
from sklearn.feature_extraction.text import HashingVectorizer
vectorizer = HashingVectorizer(n_features=999)
vectorizer.fit_transform(df['text'])

<1235x999 sparse matrix of type '<class 'numpy.float64'>'
	with 264863 stored elements in Compressed Sparse Row format>

In [18]:
X_train_hv = vectorizer.transform(X_train)
X_test_hv = vectorizer.transform(X_test)

In [27]:

svm_model = SVC()
svm_model.fit(X_train_hv, y_train)

# Đánh giá mô hình
y_pred = svm_model.predict(X_test_hv)
accuracy = accuracy_score(y_test, y_pred)
confusion_mat = confusion_matrix(y_test, y_pred)

print("Accuracy:", accuracy)
print("Confusion Matrix:\n", confusion_mat)

Accuracy: 0.9838709677419355
Confusion Matrix:
 [[26  0  1  0  0]
 [ 0 22  0  0  0]
 [ 0  0 21  0  0]
 [ 0  0  0 27  0]
 [ 0  0  1  0 26]]


In [29]:
bayes_model = GaussianNB()
bayes_model.fit(X_train_hv.toarray(), y_train)

y_pred = bayes_model.predict(X_test_hv.toarray())
accuracy = accuracy_score(y_test, y_pred)
confusion_mat = confusion_matrix(y_test, y_pred)

print("Accuracy:", accuracy)
print("Confusion Matrix:\n", confusion_mat)

Accuracy: 0.9112903225806451
Confusion Matrix:
 [[23  0  4  0  0]
 [ 1 20  0  0  1]
 [ 1  1 17  1  1]
 [ 1  0  0 26  0]
 [ 0  0  0  0 27]]
